<a href="https://colab.research.google.com/github/Prattoypaul/deeplearning/blob/machinel/ChatBotDeepLearing(DT_457).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import nltk
from nltk.stem.porter import PorterStemmer

In [3]:
def tokenize(sentence):
  return nltk.word_tokenize(sentence)

In [4]:
stemmer=PorterStemmer()

In [5]:
def stem(word):
  return stemmer.stem(word.lower())

In [6]:
def bag_of_words(tokenized_sentence,all_words):
  tokenized_sentence=[stem(w) for w in tokenized_sentence ]
  bag=np.zeros(len(all_words),dtype=np.float32)
  for indx,w in enumerate(all_words):
    if w in tokenized_sentence:
      bag[indx]=1.0
  return bag

In [7]:
sen=['hlw','how','are','you']
word=['hlw','is','how','my','are','dear','you']
bg=bag_of_words(sen,word)
bg

array([1., 0., 1., 0., 1., 0., 1.], dtype=float32)

In [8]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
a="How I can help you?"
print (a)

How I can help you?


In [10]:
words=['Organize','Organizes','Organizing']
stemmed_words = [stem(w) for w in words]
print(stemmed_words)

['organ', 'organ', 'organ']


In [11]:
import json

In [12]:
with open('/content/drive/MyDrive/Colab Notebooks/A==4 chatbot/intents.json','r' ) as f:
  intents=json.load(f)


In [13]:
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What help you provide?', 'How you can be helpful?'], 'responses': ['I can guide you through Adverse management problems, order tracking, p

In [14]:
all_words= []
tags= []
xy=[]
for intent in intents['intents']:
  tag=intent['tag']
  tags.append(tag)
  for pattern in intent['patterns']:
    w=tokenize(pattern)
    all_words.extend(w)
    xy.append((w,tag))

In [15]:
ignore_word=['!',',','?','.']
print(all_words)
print(tags)


['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Awesome', ',', 'thanks', 'Thanks', 'for', 'helping', 'me', 'How', 'you', 'could', 'help', 'me', '?', 'What', 'help', 'you', 'provide', '?', 'How', 'you', 'can', 'be', 'helpful', '?', 'where', 'is', 'order', 'with', 'id', '431B67', '?', 'track', 'order', '562B78', 'Where', 'is', 'my', 'order', 'with', 'id', '561A24', '?', 'order', 'id', '345A23', 'comprises', 'of', '?', 'List', 'of', 'components', 'where', 'is', 'the', 'order', 'where', 'is', 'my', 'order', 'locate', 'the', 'order', 'Delivery', 'date', 'of', 'order', 'find', 'order', 'location', '32712', 'What', 'is', 'the', 'Location', 'of', 'order', '23A31', '?', 'I', 'want', 'an', 'appoitment', 'with', 'Manoj', 'kumar', 'set', 'an', 'appointment', 'with', 'Sujata', 'Nand

In [16]:
all_words=[stem(w) for w in all_words if  w not in ignore_word]
all_words=sorted(set(all_words))
tags=sorted(set(tags))
print(tags)

['HR_related_problem', 'Location', 'Weather', 'about', 'appointment status', 'cabin', 'check_leave', 'commission', 'competitors_in_market', 'configuration', 'connect_people', 'cost_lowering', 'customer_satisfaction', 'domain', 'email_id', 'factors_impacting_sale', 'forgot_password', 'gadgets', 'goodbye', 'greeting', 'highest_grossing', 'hours', 'invalid', 'key_customers', 'leave', 'maintainence', 'manufacturing_problems', 'missing_id', 'name', 'noans', 'noanswer', 'options', 'order_components', 'order_tracking', 'predict_delay', 'predict_performance', 'project_handling_queries', 'search_department', 'search_person_by_id', 'solve_problems', 'supplier_info', 'thanks', 'turnover', 'version_update']


In [17]:
xtrain=[]
ytrain=[]

In [18]:
for (pattern_sentence,tag) in xy:
  bag=bag_of_words(pattern_sentence,all_words)
  xtrain.append(bag)
  label=tags.index(tag)
  ytrain.append(label)


In [19]:
import numpy as np

In [20]:
xtrain=np.array(xtrain)
ytrain=np.array(ytrain)

In [21]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

In [22]:
class ChatDataset(Dataset):
  def __init__(self):
    self.n_samples = len(xtrain)
    self.x_data=xtrain
    self.y_data=ytrain

  def __getitem__(self,index):
    return self.x_data[index],self.y_data[index]
  def __len__(self):
    return self.n_samples

In [23]:
dataset=ChatDataset()
batch=8


In [24]:
train_loader=DataLoader(dataset=dataset,batch_size=batch,shuffle=True,num_workers=2)

In [25]:
class NeuralNet(nn.Module):
  def __init__(self,input_size,hide_size,num_class):
    super(NeuralNet,self).__init__()
    self.l1=nn.Linear(input_size,hide_size)
    self.l2=nn.Linear(hide_size,hide_size)
    self.l3=nn.Linear(hide_size,num_class)
    self.relu=nn.ReLU()

  def forward(self, x):
    out =self.l1(x)
    out =self.relu(out)
    out =self.l2(out)
    out =self.relu(out)
    out =self.l3(out)


    return out
    

In [26]:
output_size=len(tags)
hide_size=8
input_size=len(xtrain[0])

In [27]:
output_size

44

In [28]:
model=NeuralNet(input_size,hide_size,output_size)

In [29]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

In [30]:
num_epochs=1000

In [31]:
for epoch in range(num_epochs):
  for (words,labels) in train_loader:
    outputs=model(words)
    loss=criterion(outputs,labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  if (epoch+1)%100==0:
    print(f'epoch {epoch+1}/{num_epochs},loss={loss.item():4f}')

epoch 100/1000,loss=0.055353
epoch 200/1000,loss=0.004587
epoch 300/1000,loss=0.000782
epoch 400/1000,loss=0.003302
epoch 500/1000,loss=0.000499
epoch 600/1000,loss=0.000088
epoch 700/1000,loss=0.000016
epoch 800/1000,loss=0.000028
epoch 900/1000,loss=0.000004
epoch 1000/1000,loss=0.758989


In [32]:
print(f'final loss,loss={loss.item():4f}')

final loss,loss=0.758989


In [33]:
data={
    'model_state': model.state_dict(),
    'input_size':input_size,
    'output_size':output_size,
    'hidden_size':hide_size,
    'all_words':all_words,
    'tags':tags 
}

In [39]:
File='data.pth'
torch.save(data,f'/content/drive/MyDrive/Colab Notebooks/A==4 chatbot/{File}')

In [40]:
import random

In [41]:
with open('/content/drive/MyDrive/Colab Notebooks/A==4 chatbot/intents.json','r') as f:
  intents=json.load(f)




In [42]:
data=torch.load('/content/drive/MyDrive/Colab Notebooks/A==4 chatbot/data.pth')

In [43]:
input_size=data['input_size']
hidden_size=data['hidden_size']
output_size=data['output_size']
all_words=data['all_words']
tags=data['tags']
model_state=data['model_state']

In [44]:
model=NeuralNet(input_size,hidden_size,output_size)
model.load_state_dict(model_state)
model.eval()

NeuralNet(
  (l1): Linear(in_features=224, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=44, bias=True)
  (relu): ReLU()
)

In [50]:
bot_name="Panda"
print("let's chat ! type 'quit' to exit")
while True:
  sentence=input("You : ")
  if sentence == 'quit':
    print(f"{bot_name}: See later:)")
    break
  sentence=tokenize(sentence)
  X=bag_of_words(sentence,all_words)
  X=X.reshape(1,X.shape[0])
  X=torch.from_numpy(X)
  output=model(X)
  _,predicted=torch.max(output,dim=1)
  tag=tags[predicted.item()]

  

  pros=torch.softmax(output,dim=1)
  prob=pros[0][predicted.item()]
  if prob.item()>0.75:
    for intent in intents['intents']:
      if tag == intent['tag']:
        print(f"{bot_name}: {random.choice(intent['responses'])}")
  else:
    print(f"{bot_name}: Sorry! I don't understand")

let's chat ! type 'quit' to exit
You : hey
Panda: Hi there, how can I help?
You : what do you sell
Panda: Thanks For Asking!How can I help you?
You : what is your service
Panda: Can't Understand Your Question
You : quit
Panda: See later:)
